Questions:
1) Does it matter what order the housing characteristics come in on the buildstock.csv file (order of columns?)
2) Should the import pandas statement be in or outside the function or will we set up an environment which already has this imported? 
3) Can we use an .xlsx file as an input? Why is everything in .csv?

In [132]:
# Import necessary libraries
import pandas as pd


In [60]:
def technology_adoption():
    # Import statements
    import pandas as pd
    from itertools import repeat
    import json
    
    # Read in buildstock file
    buildstock = pd.read_csv('buildstock.csv')
    
    # Input file to be 
    technology_file = input("Input technology adoption file name:")
    with open(technology_file) as f:
        technology_data = json.load(f)
    
    # Read in the different sheets from the technology adoption file which just has the name of the .tsv technology that we are adjusting
    technology_df = pd.json_normalize(technology_data, record_path = ["tsv"]) # tsv name
    saturation_2035 = pd.json_normalize(technology_data, record_path = ["option"]) # Technology saturations for 2035
    replacement_2035 = pd.json_normalize(technology_data, record_path = ["replacement"]) # Distribution matrix
    dependencies_df = saturation_2035.set_index('option', inplace = False) #Dataframe to address dependencies
    
    # Create a new buildstock dataframe which will serve as the basis for the new_buildstock.csv file
    new_buildstock = pd.DataFrame()
    
    # Create the variable technology, which is a string which will be used to access the correct housing characteristics column in buildstock.csv
    technology = technology_df.iat[0,0]
    
    # Determine the original technology saturations from the buildstock.csv
    original_percent = []
    for option in saturation_2035.iloc[:,0]:
        partial_df = buildstock.loc[buildstock[technology] == option]
        partial_df_size = partial_df.shape[0]/buildstock.shape[0]
        original_percent.append(partial_df_size)
    
    # Use new percent to determine the number of houses which will get a replacement
    total = buildstock.shape[0]
    for option, percent in zip(saturation_2035["option"], saturation_2035["goal percent"]):
        original = buildstock.loc[buildstock[technology] == option]
        original_size = original.shape[0]
        if original_size == 0: 
            continue 
            
        # Use new percent to determine the number of houses which will get a replacement
        goal = total*percent
        if goal >= original_size:
            keep = original_size
            keep = int(keep)
            replace = 0 
            replace = int(replace)
        else: 
            keep = goal
            keep = int(keep)
            replace = original_size-keep
            replace = int(replace)
        
        # New technology distribution
        new_option = []
        
        # The number of households which do not change
        new_option.extend(repeat(option, keep))
        
        # The number of the households which get a replacement, using the distribution matrix
        for replacement, dist in zip(replacement_2035['replacement'], replacement_2035[option]):
            new_tech = dist*replace
            new_tech = int(new_tech)
            new_option.extend(repeat(replacement, new_tech))
        
        # Ensure the new option size matches the size of the original dataframe
        diff = original_size - len(new_option)
        if diff < 0: 
            del new_option[diff:]
        elif diff > 0:
            new_option.extend(repeat(replacement_2035['replacement'][:-1], diff))
    
        # To check that code above added/removed the necessary number of entries
        new_diff = original_size - len(new_option)
        #print(new_diff)
        
        # Remove final datapoint with information about the dataframe
        new_option[-1] = option
        
        size = dependencies_df.shape[1]
        counter = 1
        columns = dependencies_df.columns
        new_dependency = []
        if size != 1: 
            while counter < size: 
                dep_loc = columns[counter]
                for option in new_option:
                    new_dep = dependencies_df.loc[option][dep_loc]
                    new_dependency.append(new_dep)
                counter += 1
        
        #new_dependency = []
        #for option in new_option:
            #new_dep = dependencies_df.loc[option]["Water Heater Fuel"]
            #print(new_dep)
            #new_dependency.append(new_dep)
            
            
        #print(new_dependency)
        # Delete original existing technology column and add in the new column
        del original[technology]
        del original["Water Heater Fuel"]
        original[technology] = new_option
        original["Water Heater Fuel"] = new_dependency
        new_buildstock = new_buildstock.append(original)
    

    # Test -> Can be removed if necessary
    # Create new technology_adoption dataframe 
    new_technology_test = pd.DataFrame()
    new_technology_test["options"] = saturation_2035.iloc[:,0] # Add options to test dataframe
    new_technology_test["original percent"] = original_percent  # Add original percent to test dataframe
    new_technology_test["goal percent"] = saturation_2035.iloc[:,1] # Add goal percent to test dataframe
    
    # Determine the new percent of each option
    new_percent = []
    for option in saturation_2035.iloc[:,0]:
        count = 0 
        for row in new_buildstock[technology]:
            if str(row) == str(option):
                count +=1
        percentage = count/total
        new_percent.append(percentage)
        

    new_technology_test["new percent"] = new_percent # Add new percent to test dataframe
    new_technology_test.to_csv("new_technology_test.csv")
    
    # Resort buildstock by building number
    new_buildstock = new_buildstock.sort_values(by="Building", axis=0, ascending=True)
    
    new_buildstock.to_csv("new_buildstock.csv") 
    
    
technology_adoption()




Input technology adoption file name:waterheater.json


/Users/nsandova/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/nsandova/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [4]:
if keep >= original_size:
            keep == original_size
            keep = int(keep)
            replace == 0
            replace = int(replace)
        else: 
            keep = int(keep)
            replace = original_size-keep
            replace = int(replace)

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 6)

In [142]:
# Read in buildstock file
buildstock = pd.read_csv('buildstock.csv')

buildstock_mini = buildstock.head(1000)

# Read in first sheet of the cookingrange.xlsx which just has the name of the .tsv technology that we are adjusting
technology_df = pd.read_excel('cookingrange.xlsx', sheet_name=0)

# Create the variable technology, which is a string which will be used to access the Cooking Range column in buildstock.csv
#technology_df = technology_df.astype({'technology':'string'})
#technology = technology_df.iat[0,0]
# Moved to cooking_range3 function


saturation_2035 = pd.read_excel('cookingrange.xlsx', sheet_name=1)
replacement_2035 = pd.read_excel('cookingrange.xlsx', sheet_name=2)

print(replacement_2035['replacement'])



0            Gas, 80% Usage
1           Gas, 100% Usage
2           Gas, 120% Usage
3        Propane, 80% Usage
4       Propane, 100% Usage
5       Propane, 120% Usage
6       Electric, 80% Usage
7      Electric, 100% Usage
8      Electric, 120% Usage
9      Induction, 80% Usage
10    Induction, 100% Usage
11    Induction, 120% Usage
12                     None
Name: replacement, dtype: object


In [143]:
def cooking_range3():
    
    # Read in first sheet of the cookingrange.xlsx which just has the name of the .tsv technology that we are adjusting
    technology_df = pd.read_excel('cookingrange.xlsx', sheet_name=0)
    
    # Create the variable technology, which is a string which will be used to access the Cooking Range column in buildstock.csv
    technology_df = technology_df.astype({'technology':'string'})
    technology = technology_df.iat[0,0]
    
    new_buildstock = pd.DataFrame()
    
    original_percent = []
    
    for option in saturation_2035.iloc[:,0]:
        partial_df = buildstock_mini.loc[buildstock_mini[technology] == option]
        partial_df_size = partial_df.shape[0]/buildstock_mini.shape[0]
        original_percent.append(partial_df_size)
    
    # Create new cookingrange dataframe which will serve as the basis for the newcookingrange.xlsx
    new_cookingrange = pd.DataFrame()
    
    new_cookingrange["options"] = saturation_2035.iloc[:,0]
    new_cookingrange["original percent"] = original_percent
    
    
    
    from itertools import repeat
    total = buildstock_mini.shape[0]
    
    for option, percent in zip(new_cookingrange["options"], saturation_2035["new percent"]):
        original = buildstock_mini.loc[buildstock_mini[technology] == option]
        original_size = original.shape[0]
        keep = total*percent
        keep = int(keep)
        replace = original_size-keep
        replace = int(replace)
        #print("For", option, 'there were originally', original, 'We keep', keep, 'and replace', replace)
        
        # New Cooking Range option distribution
        new_option = []
        
        # The number of households which keep
        new_option.extend(repeat(option, keep))
        
        for replacement, dist in zip(replacement_2035['replacement'], replacement_2035[option]):
            new_tech = dist*replace
            new_tech = int(new_tech)
            new_option.extend(repeat(replacement, new_tech))
    
    
        # Ensure the new option size matches the size of the original dataframe
        diff = original_size - len(new_option)
        if diff < 0: 
            del new_option[diff:]
        elif diff > 0:
            new_option.extend(repeat(replacement_2035['replacement'][:-1], diff))
    
        # To check that code above added/removed the necessary number of entries
        new_diff = original_size - len(new_option)
        
        del original[technology]
        original[technology] = new_option
        #print(original)
        new_buildstock = new_buildstock.append(original)
    
    # Resort buildstock by building number
    new_buildstock = new_buildstock.sort_values(by="Building", axis=0, ascending=True)
    
    new_buildstock.to_csv("new_buildstock.csv")  
    
    
        
cooking_range3()

def test():
    test = pd.read_csv('new_buildstock.csv')
    
    total = test.shape[0]
    
    # Read in first sheet of the cookingrange.xlsx which just has the name of the .tsv technology that we are adjusting
    technology_df = pd.read_excel('cookingrange.xlsx', sheet_name=0)
    
    # Create the variable technology, which is a string which will be used to access the Cooking Range column in buildstock.csv
    technology_df = technology_df.astype({'technology':'string'})
    technology = technology_df.iat[0,0]
    
    original_percent = []
    
    for option in saturation_2035.iloc[:,0]:
        partial_df = buildstock_mini.loc[buildstock_mini[technology] == option]
        partial_df_size = partial_df.shape[0]/buildstock_mini.shape[0]
        original_percent.append(partial_df_size)
        print(option)
    new_cookingrange = pd.DataFrame()
    
    new_cookingrange["options"] = saturation_2035.iloc[:,0]
    new_cookingrange["original percent"] = original_percent
    
    new_percent = []
    
    for option in new_cookingrange["options"]:
        print(option)
        original = test.loc[test[technology] == option]
        original_size = original.shape[0]
        percent = original_size/total
        new_percent.append(percent)
        
    new_cookingrange["new percent"] = new_percent
    new_cookingrange["goal"] = saturation_2035.iloc[:,1]
    
    new_cookingrange.to_csv("new_cookingrange")
        
test()

/Users/nsandova/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Gas, 80% Usage
Gas, 100% Usage
Gas, 120% Usage
Propane, 80% Usage
Propane, 100% Usage
Propane, 120% Usage
Electric, 80% Usage
Electric, 100% Usage
Electric, 120% Usage
Induction, 80% Usage
Induction, 100% Usage
Induction, 120% Usage
None
Gas, 80% Usage
Gas, 100% Usage
Gas, 120% Usage
Propane, 80% Usage
Propane, 100% Usage
Propane, 120% Usage
Electric, 80% Usage
Electric, 100% Usage
Electric, 120% Usage
Induction, 80% Usage
Induction, 100% Usage
Induction, 120% Usage
None


In [139]:
def cooking_range_2():
    from itertools import repeat
    total = buildstock_mini.shape[0]
    
    new_buildstock = pd.DataFrame()
    
    
    # New technology saturations
    tech_adoption = pd.DataFrame({'option': ["Gas, 80% Usage", "Gas, 100% Usage", "Gas, 120% Usage"], 
                    'replacement': ["Electric, 80% Usage", "Electric, 100% Usage", "Electric, 120% Usage"],
                    'percent': [0.097, 0.194, 0.097]
                    })
    
    # Update technology saturations
    for option, percent, replacement in zip(tech_adoption['option'],tech_adoption['percent'], tech_adoption['replacement']):
        var = buildstock_mini.loc[buildstock_mini['Cooking Range'] == option]
        var_goal = total*percent
        var_original = var.shape[0]
        var_diff = var_original-var_goal
        
        new_option = []
        new_option.extend(repeat(option, int(var_goal)))
        new_option.extend(repeat(replacement, int(var_diff)))
        #print(new_option)
        
        del var["Cooking Range"]
        var["Cooking Range"] = new_option
        print(type(var))
        new_buildstock = new_buildstock.append(var)
        
    
    no_change = pd.DataFrame({'option': ["Propane, 80% Usage", "Propane, 100% Usage", "Propane, 120% Usage", 
                               "Electric, 80% Usage", "Electric, 100% Usage", "Electric, 120% Usage", 
                               "Induction, 80% Usage", "Induction, 100% Usage", "Induction, 120% Usage",
                               "None"], 
                    'percent': [0.001,0.001,0.001,
                               0.146, 0.293, 0.146,
                                0,0,0,
                               0.024]
                    })
    
    for option in no_change['option']:
        var = buildstock_mini.loc[buildstock_mini['Cooking Range'] == option]
        var_original = var.shape[0]
        
        new_option = []
        new_option.extend(repeat(option, int(var_original)))
        #print(new_option)
        
        del var["Cooking Range"]
        var["Cooking Range"] = new_option
        new_buildstock = new_buildstock.append(var)
    
    # Resort buildstock by building number
    new_buildstock = new_buildstock.sort_values(by="Building", axis=0, ascending=True)
    
    print(new_buildstock)
    
    new_buildstock.to_csv("new_buildstock.csv")
    #new_technologies_percents.to_csv    
    
    
    
cooking_range_2()  

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
     Building   Schedules Usage Level Orientation Eaves Door Area Overhangs  \
0           1  Stochastic      Medium        West  2 ft   20 ft^2      None   
1           2  Stochastic      Medium   Northwest  2 ft   20 ft^2      None   
2           3  Stochastic      Medium       North  2 ft   20 ft^2      None   
3           4  Stochastic      Medium       North  2 ft   20 ft^2      None   
4           5  Stochastic      Medium   Southeast  2 ft   20 ft^2      None   
..        ...         ...         ...         ...   ...       ...       ...   
995       996  Stochastic      Medium   Southwest  2 ft   20 ft^2      None   
996       997  Stochastic      Medium        West  2 ft   20 ft^2      None   
997       998  Stochastic        High   Southwest  2 ft   20 ft^2      None   
998       999  Stochastic      Medium       North  2 ft   20 ft^2      None   
999      1000  St

/Users/nsandova/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/nsandova/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [30]:
# Function to divide households into certain demographics
def income_level():
    none = buildstock.loc[buildstock['Income'] == 'None']
    zero_ten = buildstock.loc[buildstock['Income'] == '<10000']
    ten_fifteen = buildstock.loc[buildstock['Income'] == '10000-14999']
    fifteen_twenty = buildstock.loc[buildstock['Income'] == '15000-19999']
    twenty_twentyfive = buildstock.loc[buildstock['Income'] == '20000-24999']
    twentyfive_thirty = buildstock.loc[buildstock['Income'] == '25000-29999']
    thirty_thirtyfive = buildstock.loc[buildstock['Income'] == '30000-34999']
    thirtyfive_forty = buildstock.loc[buildstock['Income'] == '35000-39999']
    forty_fortyfive = buildstock.loc[buildstock['Income'] == '40000-44999']
    fortyfive_fifty = buildstock.loc[buildstock['Income'] == '45000-49999']
    fifty_sixty = buildstock.loc[buildstock['Income'] == '50000-59999']
    sixty_seventy = buildstock.loc[buildstock['Income'] == '60000-69999']
    seventy_eighty = buildstock.loc[buildstock['Income'] == '70000-79999']
    eighty_ninety = buildstock.loc[buildstock['Income'] == '80000-89999']
    ninety_one = buildstock.loc[buildstock['Income'] == '90000-99999']
    one_onetwenty = buildstock.loc[buildstock['Income'] == '100000-119999']
    onetwenty_oneforty = buildstock.loc[buildstock['Income'] == '120000-139999']
    oneforty_onesixty = buildstock.loc[buildstock['Income'] == '140000-159999']
    onesixty_oneeighty = buildstock.loc[buildstock['Income'] == '160000-179999']
    oneeighty_two = buildstock.loc[buildstock['Income'] == '180000-199999']
    twoplus = buildstock.loc[buildstock['Income'] == '200000+']

def tenure():
    none = buildstock.loc[buildstock['Tenure'] == 'None']
    owner_occupied = buildstock.loc[buildstock['Tenure'] == 'Owner Occupied']
    renter_occupied = buildstock.loc[buildstock['Tenure'] == 'Renter Occupied']
    print(renter_occupied)
    

In [133]:
# Function does heat pump technology adoption (whatever that is)
# This function takes in the divided households above. 
def cooking_range():
    from itertools import repeat
    
    # New technology saturations
    tech_adoption = pd.DataFrame({'option': ["Gas, 80% Usage", "Gas, 100% Usage", "Gas, 120% Usage",
                               "Propane, 80% Usage", "Propane, 100% Usage", "Propane, 120% Usage", 
                               "Electric, 80% Usage", "Electric, 100% Usage", "Electric, 120% Usage", 
                               "Induction, 80% Usage", "Induction, 100% Usage", "Induction, 120% Usage",
                               "None"], 
                    'percent': [0.097, 0.194, 0.097,
                               0.001,0.001,0.001,
                               0.146, 0.293, 0.146,
                                0,0,0,
                               0.024]
                    })
    
    # Randomize buildstock file
    buildstock_shuffle = buildstock_mini.sample(frac = 1)
    
    # Add column to tech_adoption
    tech_adoption['number'] = tech_adoption['percent']*buildstock_shuffle.shape[0]
    
    # Create a new series of data with the new technology saturation numbers
    new_option=[]
    for number, option in zip(tech_adoption['number'],tech_adoption['option']):
        new_option.extend(repeat(option,int(number)))
    
    # Delete original cooking range column with new one
    del buildstock_shuffle["Cooking Range"]
    buildstock_shuffle["Cooking Range"] = new_option
    
    # Resort buildstock by building number
    new_buildstock = buildstock_shuffle.sort_values(by="Building", axis=0, ascending=True)
    
    new_buildstock.to_csv("new_buildstock.csv")

In [134]:
# Call functions
cooking_range()

     Building   Schedules Usage Level Orientation Eaves Door Area Overhangs  \
20         21  Stochastic      Medium       South  2 ft   20 ft^2      None   
968       969  Stochastic         Low   Southwest  2 ft   20 ft^2      None   
924       925  Stochastic        High        West  2 ft   20 ft^2      None   
811       812  Stochastic         Low       North  2 ft   20 ft^2      None   
95         96  Stochastic      Medium        West  2 ft   20 ft^2      None   
..        ...         ...         ...         ...   ...       ...       ...   
459       460  Stochastic      Medium        West  2 ft   20 ft^2      None   
665       666  Stochastic        High        West  2 ft   20 ft^2      None   
149       150  Stochastic         Low        West  2 ft   20 ft^2      None   
711       712  Stochastic      Medium        East  2 ft   20 ft^2      None   
789       790  Stochastic      Medium        West  2 ft   20 ft^2      None   

                Interior Shading       Doors Hot Wa

In [130]:
# Write new buildstock file
new_buildstock.to_csv("new_buildstock.csv")

NameError: name 'new_buildstock' is not defined